# Automated Question-Answer Generation from Pharmaceutical Data

This notebook systematically generates question-answer pairs from pharmaceutical data sources including:
1. PubMed target passages
2. DrugBank tables
3. Related pharmaceutical data

Each QA pair will include:
- Question
- Answer
- Source text/passage
- Related table

In [1]:
# Import required libraries
import os
import pandas as pd
import json
from pathlib import Path
from openai import OpenAI
import csv
from tqdm import tqdm
from collections import defaultdict
import time
import re


In [2]:
# Configuration and paths
PUBMED_TARGETS_DIR = '../data/Pharma/pubmed-targets'
DRUGBANK_TABLES_DIR = '../data/Pharma/drugbank-tables'
MAPPING_FILE = '../data/Pharma/pubmed-drugbank-tables.gt'
OUTPUT_FILE = 'passage_output_2.gt'

# Initialize your LLM API key if needed
# os.environ["OPENAI_API_KEY"]="<KEY>"
client = OpenAI()


In [3]:
def load_passage_table_mapping():
    """Load the mapping between passages and their relevant tables"""
    mapping = defaultdict(list)
    with open(MAPPING_FILE, 'r') as f:
        for line in f:
            passage_id, table_name = line.strip().split(',')
            mapping[passage_id].append(table_name)
    # print("mapping", mapping)
    return mapping

def load_target_passages():
    """Load all target passages from the pubmed-targets directory"""
    passages = {}
    target_files = Path(PUBMED_TARGETS_DIR).glob('Target-*')
    
    for file_path in target_files:
        target_id = file_path.name
        with open(file_path, 'r') as f:
            passages[target_id] = f.read()
    # print("passages", passages)
    return passages

def load_drugbank_tables():
    """Load all relevant DrugBank tables"""
    tables = {}
    csv_files = Path(DRUGBANK_TABLES_DIR).glob('*.csv')
    
    for file_path in csv_files:
        table_name = file_path.stem
        tables[table_name] = pd.read_csv(file_path)
    
    # print("tables", tables)
    return tables

In [4]:
def generate_questions_for_passages(passage_ids, passage_texts, model="gpt-4o"):
    """Generate questions for a given passage and its relevant tables using LLM"""
    
    # Format all passages with their IDs for the prompt
    passages_formatted = ""
    for idx, (pid, text) in enumerate(zip(passage_ids, passage_texts)):
        passages_formatted += f"Passage {idx+1} (ID: {pid}):\n{text}\n\n"

    # have the system generate one question answer pair that is very difficult/numerical and then taken another passage and aask the LLM to generate another question answer pair, thne try and combine the two questions and answers
    prompt = f"""
    Given the following passages, generate 1 meaningful question-answer pair.
    IMPORTANT: Each question MUST require synthesizing information from ALL passages to answer correctly.
    Focus on pharmaceutical and medical aspects. Make the question as difficult and technical as possible.
    
    {passages_formatted}
    
    Generate questions in the following format:
    1. question: [specific question requiring information from all passages]
       answer: [detailed answer synthesizing information from all passages]
       text: [{', '.join(passage_ids)}]
       table: [None]
    
    
    """

    # Ensure the question CANNOT be answered using just one passage - it must require connecting information across all provided passages.
    
    # Call your LLM here with the prompt
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a medical and pharmaceutical expert tasked with generating detailed question-answer pairs about drugs and treatments that require synthesizing information from multiple sources."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )

    # print(response.choices[0].message.content)
    
    # Parse the response into structured QA pairs
    qa_pairs = parse_llm_response(response.choices[0].message.content, ", ".join(passage_ids))
    return qa_pairs

def parse_llm_response(response_text, passage_ids):
    """Parse the LLM response into structured QA pairs"""
    qa_pairs = []
    
    # Split the response into individual QA entries
    entries = response_text.strip().split('\n\n')
    
    for entry in entries:
        if not entry.strip():
            continue
            
        lines = entry.strip().split('\n')
        current_qa = {
            'question': '',
            'answer': '',
            'text': passage_ids,
            'table': 'None'  # Default value
        }
        
        for line in lines:
            line = line.strip()
            # Skip empty lines and numbering
            if not line or line.replace('.', '').strip().isdigit():
                continue
                
            # Parse each field using more robust splitting
            if 'question:' in line:
                current_qa['question'] = line.split('question:', 1)[1].strip()
            elif 'answer:' in line:
                current_qa['answer'] = line.split('answer:', 1)[1].strip()
            elif 'text:' in line:
                current_qa['text'] = line.split('text:', 1)[1].strip()
            elif 'table:' in line:
                table_value = line.split('table:', 1)[1].strip()
                # Handle NA, N/A, None cases
                current_qa['table'] = 'None' if table_value.upper() in ['NA', 'N/A', 'NONE'] else table_value
        
        # Only add complete QA pairs that have both question and answer
        if current_qa['question'] and current_qa['answer']:
            qa_pairs.append(current_qa.copy())  # Use copy to avoid reference issues
    
    return qa_pairs

In [5]:
def verify_qa_uses_sources(qa_pair, content, model="gpt-4o"):
    # Create prompt for the LLM
    prompt = f"""
    Your task is to determine if the provided answer to a question uses information from the passages.
    
    Question: {qa_pair['question']}
    Answer: {qa_pair['answer']}
    
    Passage Content:
    {content}
    
    Does the answer draw specific information from the passages? Reply with just "YES" or "NO".
    If the answer only uses requires information ourtside of the content provided in the passages, reply "NO".
    """
    
    # Call the LLM for verification
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a verification expert who determines if answers draw from multiple data sources."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.1
    )
    
    verification = response.choices[0].message.content.strip().upper()
    return verification
    

In [6]:
def main():
    print("Loading target passages...")
    passages = load_target_passages()
    
    # Load drug grouping file
    with open('../grouped_passages_by_drug.json', 'r') as f:
        drug_to_passages = json.load(f)
    
    # Initialize output list
    qa_pairs = []
    
    # Get a list of drug types with multiple passages
    drug_types = []
    for drug, passage_list in drug_to_passages.items():
        if len(passage_list) >= 2:  # Only consider drugs with at least 2 passages
            drug_types.append(drug)
    
    # Limit to a manageable number of drug types (e.g., 10)
    selected_drug_types = drug_types[:10]
    
    for drug_type in selected_drug_types:
        # Get passages for this drug type
        drug_passages = drug_to_passages[drug_type]
        
        # Take a pair of different passages about this drug
        if len(drug_passages) >= 2:
            for i in range(0, min(4, len(drug_passages)), 2):  # Process up to 2 pairs per drug
                if i+1 < len(drug_passages):  # Make sure we have a pair
                    pair_ids = [drug_passages[i], drug_passages[i+1]]
                    pair_texts = [passages[pid] for pid in pair_ids]
                    
                    print(f"Generating questions for {drug_type} using passages: {pair_ids}")
                    
                    # Generate QA pairs using the pair of passages
                    new_qa_pairs = generate_questions_for_passages(
                        pair_ids,
                        pair_texts
                    )

                    for qa_pair in new_qa_pairs:
                        # Combine passage texts for verification
                        combined_passage_text = "\n\n".join(pair_texts)
                        verification = verify_qa_uses_sources(qa_pair, combined_passage_text)
                        if verification == "YES":
                            qa_pairs.append(qa_pair)
                        else:
                            print(f"Question: {qa_pair['question']} - Answer: {qa_pair['answer']} - Verification: {verification}")
    
    # Save results
    with open(OUTPUT_FILE, 'w', newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_ALL)
        writer.writerow(['question', 'answer', 'text', 'table'])  # header
        for qa_pair in qa_pairs:
            writer.writerow([
                qa_pair['question'],
                qa_pair['answer'],
                qa_pair['text'],
                qa_pair['table']
            ])
        
    print(f"Generated {len(qa_pairs)} question-answer pairs")

In [22]:
# def main():
    
#     print("Loading target passages...")
#     passages = load_target_passages()
    
#     # Initialize output list
#     qa_pairs = []

#     # Convert passages dictionary to lists
#     passage_ids = list(passages.keys())[:20]
#     passage_texts = [passages[pid] for pid in passage_ids]
    
    
#     # Process passages in pairs
#     for i in range(0, len(passage_ids), 2):
#     # for i in range(0, len(passage_ids)):
#         # if i+2 < len(passage_ids):  # Make sure we have a pair
#         if i < len(passage_ids):  # Make sure we have a pair
#             pair_ids = [passage_ids[i]]
#             pair_texts = [passage_texts[i]]
            
#             # Generate QA pairs using the pair of passages
#             new_qa_pairs = generate_questions_for_passages(
#                 pair_ids,
#                 pair_texts
#             )

#             for qa_pair in new_qa_pairs:
#                 verification = verify_qa_uses_sources(qa_pair, passage_texts[i])
#                 if verification == "YES":
#                     qa_pairs.append(qa_pair)
#                 else:
#                     print(f"Question: {qa_pair['question']} - Answer: {qa_pair['answer']} - Verification: {verification}")

#             # print(new_qa_pairs)
#             # qa_pairs.extend(new_qa_pairs)
    
    
#     # Save results
#     with open(OUTPUT_FILE, 'w', newline='') as f:
#         writer = csv.writer(f, quoting=csv.QUOTE_ALL)
#         writer.writerow(['question', 'answer', 'text', 'table'])  # header
#         for qa_pair in qa_pairs:
#             writer.writerow([
#                 qa_pair['question'],
#                 qa_pair['answer'],
#                 qa_pair['text'],
#                 qa_pair['table']
#             ])
        
#     print(f"Generated {len(qa_pairs)} question-answer pairs")

In [31]:
if __name__ == "__main__":
    main()

Loading target passages...
Generating questions for Leucine using passages: ['Target-10193776', 'Target-10322928']
1. question: How does the presence of cardiovascular conditions such as congestive heart failure (CHF) and acute myocardial infarction (AMI) potentially alter the pharmacokinetic profile of cerivastatin through changes in plasma protein binding, and what implications does this have for its efficacy in reducing atherosclerotic lesions?
   answer: The presence of cardiovascular conditions such as CHF and AMI can alter the pharmacokinetic profile of cerivastatin through changes in plasma protein binding, specifically involving alpha 1-acid glycoprotein (AGP). In patients with CHF and AMI, AGP levels are elevated, which can affect the binding of drugs to plasma proteins, potentially altering their free concentrations in the serum. Although cerivastatin primarily lowers cholesterol levels and stabilizes atherosclerotic lesions by reducing macrophage infiltration and extracellul

In [32]:

def load_ground_truth(gt_file: str):
    """Load ground truth data from CSV file."""
    gt_data = []
    with open(gt_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            gt_data.append(row)
    return gt_data

def generate_llm_answer(question: str, model: str = "gpt-4o"):
    """
    Generate an answer to a question using an LLM.
    
    Args:
        question: The question to answer
        model: Model to use for generation
        
    Returns:
        The generated answer
    """
    try:
        # Create a simple prompt with just the question
        prompt = f"Answer this question with detailed information: {question}"
        print(f"Prompt: {prompt}")
        
        # Call OpenAI API
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0,  # Use low temperature for more consistent answers
            max_tokens=500
        )
        
        # Extract the response text
        return response.choices[0].message.content.strip()
            
    except Exception as e:
        print(f"Error generating answer: {e}")
        time.sleep(2)  # Sleep to handle rate limits
        return ""

def calculate_llm_correctness(hypothesis: str, reference: str, question: str, model: str = "gpt-4o"):
    """
    Use an LLM to evaluate the correctness of the hypothesis compared to the reference.
    
    Args:
        hypothesis: The system-generated answer
        reference: The ground truth answer
        question: The original question
        model: Model to use for evaluation
        
    Returns:
        A score between 0 and 1 representing correctness (1 = fully correct, 0 = incorrect)
    """
    try:
        # Create prompt for the LLM
        prompt = f"""
You are an expert evaluator assessing the correctness of an answer to a question.

Question: {question}

Ground Truth Answer: {reference}

System Answer: {hypothesis}

Evaluate how correct the System Answer is compared to the Ground Truth Answer. Be very critical in your evaluation/analysis.
Give a score from 0 to 1 where:
- 1.0 means the System Answer is fully correct and contains all the information from the Ground Truth
- 0.0 means the System Answer is completely incorrect
- Values between 0 and 1 indicate partial correctness

Output a single line with just the score as a decimal between 0 and 1.
"""

        # Call OpenAI API
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0,  # Use low temperature for more consistent evaluations
            max_tokens=300
        )
        
        # Extract the response text
        response_text = response.choices[0].message.content.strip()
        
        # Extract the score from the response - find the last number between 0 and 1
        score_matches = re.findall(r'(?:^|\s)(0(?:\.\d+)?|1(?:\.0+)?)(?:$|\s)', response_text)
        if score_matches:
            score = float(score_matches[-1])  # Take the last match as the final score
            return min(max(score, 0.0), 1.0)  # Ensure score is between 0 and 1
        else:
            print(f"Could not extract a score from LLM response: {response_text}")
            return 0.0
            
    except Exception as e:
        print(f"Error in LLM evaluation: {e}")
        time.sleep(2)  # Sleep to handle rate limits
        return 0.0

def filter_qa_pairs(gt_data,  llm_model: str = "gpt-4o", threshold: float = 0.5):
    """
    Filter question-answer pairs based on LLM correctness score.
    
    Args:
        gt_data: List of dictionaries containing ground truth data
        llm_model: Model to use for generation and evaluation
        threshold: Correctness threshold above which pairs are removed
        
    Returns:
        Filtered list of dictionaries
    """
    filtered_data = []
    
    for qa_pair in tqdm(gt_data, desc="Filtering QA pairs"):
        question = qa_pair['question']
        reference_answer = qa_pair['answer']
        
        # Generate an answer using just the question
        llm_answer = generate_llm_answer(question)
        
        # If we couldn't get an answer, keep the pair
        if not llm_answer:
            filtered_data.append(qa_pair)
            continue
        
        # Evaluate the correctness
        correctness_score = calculate_llm_correctness(
            llm_answer, reference_answer, question
        )
        
        print(f"Question: {question}")
        print(f"LLM Answer: {llm_answer[:100]}...")
        print(f"Correctness Score: {correctness_score}")
        
        # Keep pairs that the LLM couldn't answer correctly
        if correctness_score <= threshold:
            filtered_data.append(qa_pair)
    
    return filtered_data

def save_filtered_data(filtered_data, output_file):
    """Save filtered data to a CSV file."""
    if not filtered_data:
        print("No data to save!")
        return
    
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=filtered_data[0].keys(), quoting=csv.QUOTE_ALL)
        writer.writeheader()
        for row in filtered_data:
            writer.writerow(row)
    


In [33]:
# Load ground truth data
print(f"Loading ground truth data")
gt_data = load_ground_truth(OUTPUT_FILE)
print(f"Loaded {len(gt_data)} QA pairs")

# Filter pairs
filtered_data = filter_qa_pairs(gt_data, 0.5)
print(f"Filtered to {len(filtered_data)} QA pairs")

filtered_output_file = OUTPUT_FILE.replace(".gt", "_filtered.gt")
# Save filtered data
save_filtered_data(filtered_data, filtered_output_file)
print(f"Saved filtered data to {filtered_output_file}")

# generate_llm_answer("What is the main target of the drug?")

Loading ground truth data
Loaded 15 QA pairs


Filtering QA pairs:   0%|          | 0/15 [00:00<?, ?it/s]

Prompt: Answer this question with detailed information: How does the presence of cardiovascular conditions such as congestive heart failure (CHF) and acute myocardial infarction (AMI) potentially alter the pharmacokinetic profile of cerivastatin through changes in plasma protein binding, and what implications does this have for its efficacy in reducing atherosclerotic lesions?


Filtering QA pairs:   7%|▋         | 1/15 [00:09<02:15,  9.65s/it]

Question: How does the presence of cardiovascular conditions such as congestive heart failure (CHF) and acute myocardial infarction (AMI) potentially alter the pharmacokinetic profile of cerivastatin through changes in plasma protein binding, and what implications does this have for its efficacy in reducing atherosclerotic lesions?
LLM Answer: The pharmacokinetic profile of cerivastatin, like many other drugs, can be significantly influenced ...
Correctness Score: 0.7
Prompt: Answer this question with detailed information: How do verapamil and torasemide differ in their mechanisms of action on calcium channels and vascular smooth muscle cells, and what implications might these differences have for their respective roles in treating hypertension?


...

Question: How does the interaction of antiviral nucleoside analogs and diuretics with the organic anion transporter 1 (OAT1) in Xenopus laevis oocytes contribute to their pharmacokinetics, and what role does probenecid play in modulating this interaction?
LLM Answer: The interaction of antiviral nucleoside analogs and diuretics with the organic anion transporter 1 (...
Correctness Score: 0.8
Prompt: Answer this question with detailed information: How does the co-administration of oral treprostinil and bosentan affect the pharmacokinetics of treprostinil, and what additional benefits might treprostinil offer in terms of cellular activity in PAH patients?


Filtering QA pairs:  93%|█████████▎| 14/15 [02:05<00:07,  7.85s/it]

Question: How does the co-administration of oral treprostinil and bosentan affect the pharmacokinetics of treprostinil, and what additional benefits might treprostinil offer in terms of cellular activity in PAH patients?
LLM Answer: The co-administration of oral treprostinil and bosentan in patients with pulmonary arterial hyperten...
Correctness Score: 0.3
Prompt: Answer this question with detailed information: How might the co-administration of celecoxib and treprostinil diethanolamine affect prostanoid release and contractility in conditions involving ureteral obstruction and pulmonary arterial hypertension, considering their metabolic pathways and effects on prostanoid synthesis?


Filtering QA pairs: 100%|██████████| 15/15 [02:14<00:00,  8.94s/it]

Question: How might the co-administration of celecoxib and treprostinil diethanolamine affect prostanoid release and contractility in conditions involving ureteral obstruction and pulmonary arterial hypertension, considering their metabolic pathways and effects on prostanoid synthesis?
LLM Answer: The co-administration of celecoxib and treprostinil diethanolamine can have significant effects on p...
Correctness Score: 0.7
Filtered to 4 QA pairs
Saved filtered data to passage_output_2_filtered.gt
